In [54]:
import tensorflow

from sklearn.preprocessing import normalize
import pandas as pd
import numpy as np
import tensorflow as tf
import random
#log_files_path = 'Data/Wavelet/Merged/'
log_files_path = 'Output/WT1layer17-13/'


#To adjust each run: log_files_path 2x (+ iloc if needed)
# n_code: 1x
# name of output file



#iloc makes sure we exclude our dependent variables/output
df=pd.read_csv(log_files_path + '1layer17-13.csv')
d = df.values
ntotal=len(df)

d = normalize(d, axis=0, norm='l2')

resultu = []
np.random.rand(4)
# Return 100 results (for instance)
for i in range(ntotal):
    
    res = random.random()
    if res < 0.1:
        resultu.append(1)
    elif res < 0.2 and res>=0.1:
        resultu.append(2)
    elif res < 0.3 and res>=0.2:
        resultu.append(3)
    elif res < 0.4 and res>=0.3:
        resultu.append(4)
    elif res < 0.5 and res>=0.4:
        resultu.append(5)
    elif res < 0.6 and res>=0.5:
        resultu.append(6)
    elif res < 0.7 and res>=0.6:
        resultu.append(8)
    elif res < 0.8 and res>=0.7:
        resultu.append(8)
    else:
        resultu.append(9)
resultu=np.array(resultu)
trainset=[]
for i in range(1,7):
    toinsert=d[resultu==i].astype(np.float32)
    trainset.append(toinsert)
validationset=d[resultu==8].astype(np.float32)
testset=d[resultu==9].astype(np.float32)


# Following Hinton-Salakhutdinov Architecture

n_code = 9

# Parameters
learning_rate = 0.005

#batch_size = 7
display_step = 1

total_batch=6
training_epochs = 5000

In [55]:
#we store the variables here
log_files_path = 'Output/WT1layer13-9/'
input_size = df.shape[1]
def layer_batch_normalization(x, n_out, phase_train):
    """
    Defines the network layers
    input:
        - x: input vector of the layer
        - n_out: integer, depth of input maps - number of sample in the batch 
        - phase_train: boolean tf.Varialbe, true indicates training phase
    output:
        - batch-normalized maps   
    
    
    """

    beta_init = tf.constant_initializer(value=0.0, dtype=tf.float32)
    beta = tf.get_variable("beta", [n_out], initializer=beta_init)
    
    gamma_init = tf.constant_initializer(value=1.0, dtype=tf.float32)
    gamma = tf.get_variable("gamma", [n_out], initializer=gamma_init)

    #tf.nn.moment: https://www.tensorflow.org/api_docs/python/tf/nn/moments
    #calculate mean and variance of x
    batch_mean, batch_var = tf.nn.moments(x, [0], name='moments')
    
    #tf.train.ExponentialMovingAverage:
    #https://www.tensorflow.org/api_docs/python/tf/train/ExponentialMovingAverage
    #Maintains moving averages of variables by employing an exponential decay.
    ema = tf.train.ExponentialMovingAverage(decay=0.9)
    ema_apply_op = ema.apply([batch_mean, batch_var])
    ema_mean, ema_var = ema.average(batch_mean), ema.average(batch_var)
    
    def mean_var_with_update():
        with tf.control_dependencies([ema_apply_op]):
            return tf.identity(batch_mean), tf.identity(batch_var)
        
    #tf.cond: https://www.tensorflow.org/api_docs/python/tf/cond
    #Return true_fn() if the predicate pred is true else false_fn()
    mean, var = tf.cond(phase_train, mean_var_with_update, lambda: (ema_mean, ema_var))
    
    
    #Here, we changesd the shape of x into [[[x1]],[[x2] ],....]

    reshaped_x = tf.reshape(x, [-1, 1, 1, n_out])
    normed = tf.nn.batch_norm_with_global_normalization(reshaped_x, mean, var, beta, gamma, 1e-3, True)
    
    return tf.reshape(normed, [-1, n_out])

def layer(x, weight_shape, bias_shape, phase_train):
    
    """
    Defines the network layers
    input:
        - x: input vector of the layer
        - weight_shape: shape the the weight maxtrix
        - bias_shape: shape of the bias vector
        - phase_train: boolean tf.Varialbe, true indicates training phase
    output:
        - output vector of the layer after the matrix multiplication and non linear transformation
    """
    
    #initialize weights
    weight_init = tf.random_normal_initializer(stddev=(1.0/weight_shape[0])**0.5)
    W = tf.get_variable("W", weight_shape, initializer=weight_init)
    
    bias_init = tf.constant_initializer(value=0)
    b = tf.get_variable("b", bias_shape, initializer=bias_init)

    logits = tf.matmul(x, W) + b
    
    #apply the non-linear function after the batch normalization
    return tf.nn.sigmoid(layer_batch_normalization(logits, weight_shape[1], phase_train))
def encoder(x, n_code, phase_train):
    
    """
    Defines the network encoder part
    input:
        - x: input vector of the encoder
        - n_code: number of neurons in the code layer (output of the encoder - input of the decoder)
        - phase_train: boolean tf.Varialbe, true indicates training phase
    output:
        - output vector: reduced dimension
    """
    
    with tf.variable_scope("encoder"):
        
        
        with tf.variable_scope("code"):
            output = layer(x, [input_size, n_code], [n_code], phase_train)


    return output

def decoder(x, n_code, phase_train):
    """
    Defines the network encoder part
    input:
        - x: input vector of the decoder - reduced dimension vector
        - n_code: number of neurons in the code layer (output of the encoder - input of the decoder) 
        - phase_train: boolean tf.Varialbe, true indicates training phase
    output:
        - output vector: reconstructed dimension of the initial vector
    """
    
    with tf.variable_scope("decoder"):
        
   
        with tf.variable_scope("output"):
            output = layer(x, [ n_code, input_size], [input_size], phase_train)

    return output

def loss(output, x):
    """
    Compute the loss of the auto-encoder
    
    intput:
        - output: the output of the decoder
        - x: true value of the sample batch - this is the input of the encoder
        
        the two have the same shape (batch_size * num_of_classes)
    output:
        - loss: loss of the corresponding batch (scalar tensor)
    
    """
    
    with tf.variable_scope("training"):
        
        l2 = tf.sqrt(tf.reduce_sum(tf.square(tf.subtract(output, x)), 1))
        train_loss = tf.reduce_mean(l2)
        train_summary_op = tf.summary.scalar("train_cost", train_loss)
        return train_loss, train_summary_op
def training(cost, global_step):
    """
    defines the necessary elements to train the network
    
    intput:
        - cost: the cost is the loss of the corresponding batch
        - global_step: number of batch seen so far, it is incremented by one 
        each time the .minimize() function is called
    """
    
    optimizer = tf.train.AdamOptimizer(learning_rate=0.001, beta1=0.9, beta2=0.999, epsilon=1e-08, use_locking=False, name='Adam')
    #optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.005)
    train_op = optimizer.minimize(cost, global_step=global_step)
    return train_op
def evaluate(output, x):
    """
    evaluates the accuracy on the validation set 
    input:
        -output: prediction vector of the network for the validation set
        -x: true value for the validation set
    output:
        - val_loss: loss of the autoencoder
        - in_image_op: input image 
        - out_image_op:reconstructed image 
        - val_summary_op: summary of the loss
    """
    
    with tf.variable_scope("validation"):
        
        #in_image_op = image_summary("input_image", x)
        
        #out_image_op = image_summary("output_image", output)
        
        l2_norm = tf.sqrt(tf.reduce_sum(tf.square(tf.subtract(output, x, name="val_diff")), 1))
        
        val_loss = tf.reduce_mean(l2_norm)
        
        val_summary_op = tf.summary.scalar("val_cost", val_loss)
        
        #return val_loss, in_image_op, out_image_op, val_summary_op
        return val_loss,  val_summary_op
"""
def image_summary(label, tensor):
    #tf.summary.image: https://www.tensorflow.org/api_docs/python/tf/summary/image
    #Outputs a Summary protocol buffer with images.

    tensor_reshaped = tf.reshape(tensor, [-1, 19, 1, 1])
    return tf.summary.image(label, tensor_reshaped)
"""
if __name__ == '__main__':
    print('we begin')
    
    with tf.Graph().as_default():

        with tf.variable_scope("autoencoder_model"):

            #the input variables are first define as placeholder 
            # a placeholder is a variable/data which will be assigned later 
            # image vector & label, phase_train is a boolean 
            x = tf.placeholder("float", [None, input_size]) # MNIST data image of shape 28*28=784
            
            phase_train = tf.placeholder(tf.bool)
            
            #define the encoder 
            code = encoder(x, n_code, phase_train)
            
            #define the decoder
            output = decoder(code, n_code, phase_train)
            
            #compute the loss 
            cost, train_summary_op = loss(output, x)

            #initialize the value of the global_step variable 
            # recall: it is incremented by one each time the .minimise() is called
            global_step = tf.Variable(0, name='global_step', trainable=False)

            train_op = training(cost, global_step)

            #evaluate the accuracy of the network (done on a validation set)
            #eval_op, in_image_op, out_image_op, val_summary_op = evaluate(output, x)
            eval_op, val_summary_op = evaluate(output, x)

            summary_op = tf.summary.merge_all()

            #save and restore variables to and from checkpoints.
            #saver = tf.train.Saver(max_to_keep=200)
            saver = tf.train.Saver()

            #defines a session
            sess = tf.Session()

            # summary writer
            #https://www.tensorflow.org/api_docs/python/tf/summary/FileWriter
            #train_writer = tf.summary.FileWriter(log_files_path+'mnist_autoencoder_hidden_' + n_code + '_logs/', graph=sess.graph)

            #val_writer   = tf.summary.FileWriter(log_files_path+'mnist_autoencoder_hidden_' + n_code + '_logs/', graph=sess.graph)

            #initialization of the variables
            init_op = tf.global_variables_initializer()

            sess.run(init_op)
            currentmin=10000000000
            currentmin_index=-1
            error_path=[]
            togive2=[]
            # Training cycle
            for epoch in range(training_epochs):

                avg_cost = 0.
                #total_batch = int(numberofsamples/batch_size)
                
                # Loop over all batches
                for i in range(total_batch):
                    
                    minibatch_x= trainset[i]
                    
                    # Fit training using batch data
                    #the training is done using the training dataset
                    _, new_cost, train_summary = sess.run([train_op, cost, train_summary_op], feed_dict={x: minibatch_x, phase_train: True})
                    
                    #train_writer.add_summary(train_summary, sess.run(global_step))
                    
                    # Compute average loss
                    avg_cost += new_cost/total_batch
                    
                
                # Display logs per epoch step
                if epoch % display_step == 0:
                    print("Epoch:", '%04d' % (epoch+1), "cost =", "{:0.9f}".format(avg_cost))

                    #the accuracy is evaluated using the validation dataset
                    #train_writer.add_summary(train_summary, sess.run(global_step))

                    #validation_loss, in_image, out_image, val_summary = sess.run([eval_op, in_image_op, out_image_op, val_summary_op], feed_dict={x: validationset, phase_train: False})
                    validation_loss,  val_summary = sess.run([eval_op, val_summary_op], feed_dict={x: validationset, phase_train: False})
                    #val_writer.add_summary(in_image, sess.run(global_step))
                    #val_writer.add_summary(out_image, sess.run(global_step))
                    #val_writer.add_summary(val_summary, sess.run(global_step))
                    print("Validation Loss:", validation_loss)
                    error_path.append(validation_loss)
                    
                    if validation_loss<currentmin:
                        currentmin=validation_loss
                        currentmin_index=epoch
                        saver.save(sess, log_files_path + 'tmp/model.ckpt')
                        togive8=[]
                        for i in range(len(d)):
                            any_image = d[i].reshape(-1,input_size)
                            output_any_image = sess.run(code,feed_dict={x:any_image,phase_train: False})
                            togive8.append(output_any_image)
                            
                        togive2=togive8
                    #save to use later
                    #https://www.tensorflow.org/api_docs/python/tf/train/Saver
                    #saver.save(sess, log_files_path+'model-checkpoint', global_step=global_step)
                    #saver.save(sess, 'C:/Users/yy2895/Desktop/st19-14-19/tmp/model.ckpt')
                    

            print("Optimization Finished!")

            test_loss = sess.run(eval_op, feed_dict={x: testset, phase_train: False})
            #nps_loss = sess.run(eval_op, feed_dict={x: mnist.validation.images, phase_train: False})
            print("Test Loss:", test_loss)
            

we begin
Epoch: 0001 cost = 1.731922785
Validation Loss: 2.7339337
Epoch: 0002 cost = 1.724924465
Validation Loss: 2.5111387
Epoch: 0003 cost = 1.718155007
Validation Loss: 2.2765498
Epoch: 0004 cost = 1.711581548
Validation Loss: 2.0003674
Epoch: 0005 cost = 1.705176850
Validation Loss: 1.8818438
Epoch: 0006 cost = 1.698922555
Validation Loss: 1.8454702
Epoch: 0007 cost = 1.692803462
Validation Loss: 1.8060355
Epoch: 0008 cost = 1.686802049
Validation Loss: 1.7761811
Epoch: 0009 cost = 1.680902600
Validation Loss: 1.7583474
Epoch: 0010 cost = 1.675087472
Validation Loss: 1.7511283
Epoch: 0011 cost = 1.669342617
Validation Loss: 1.7368945
Epoch: 0012 cost = 1.663655400
Validation Loss: 1.7177093
Epoch: 0013 cost = 1.658014576
Validation Loss: 1.6828554
Epoch: 0014 cost = 1.652412136
Validation Loss: 1.6629275
Epoch: 0015 cost = 1.646840771
Validation Loss: 1.6594201
Epoch: 0016 cost = 1.641294797
Validation Loss: 1.6380627
Epoch: 0017 cost = 1.635772010
Validation Loss: 1.6264188
Epoch

Epoch: 0146 cost = 1.004900893
Validation Loss: 0.99655193
Epoch: 0147 cost = 1.000788003
Validation Loss: 0.9876924
Epoch: 0148 cost = 0.996689091
Validation Loss: 0.9746142
Epoch: 0149 cost = 0.992604375
Validation Loss: 0.9829864
Epoch: 0150 cost = 0.988533398
Validation Loss: 0.9886884
Epoch: 0151 cost = 0.984476060
Validation Loss: 0.98477036
Epoch: 0152 cost = 0.980433196
Validation Loss: 0.97474194
Epoch: 0153 cost = 0.976403773
Validation Loss: 0.97002655
Epoch: 0154 cost = 0.972388575
Validation Loss: 0.96613634
Epoch: 0155 cost = 0.968386948
Validation Loss: 0.96824646
Epoch: 0156 cost = 0.964399725
Validation Loss: 0.9627322
Epoch: 0157 cost = 0.960425933
Validation Loss: 0.9576969
Epoch: 0158 cost = 0.956466049
Validation Loss: 0.95681214
Epoch: 0159 cost = 0.952519953
Validation Loss: 0.9542935
Epoch: 0160 cost = 0.948588173
Validation Loss: 0.94295925
Epoch: 0161 cost = 0.944670101
Validation Loss: 0.9423784
Epoch: 0162 cost = 0.940765947
Validation Loss: 0.93891937
Epoch

Epoch: 0288 cost = 0.552759707
Validation Loss: 0.5530288
Epoch: 0289 cost = 0.550415953
Validation Loss: 0.5489694
Epoch: 0290 cost = 0.548081835
Validation Loss: 0.5424353
Epoch: 0291 cost = 0.545757870
Validation Loss: 0.5415572
Epoch: 0292 cost = 0.543443640
Validation Loss: 0.54049885
Epoch: 0293 cost = 0.541139454
Validation Loss: 0.54266423
Epoch: 0294 cost = 0.538844764
Validation Loss: 0.5440771
Epoch: 0295 cost = 0.536559681
Validation Loss: 0.5427625
Epoch: 0296 cost = 0.534284631
Validation Loss: 0.5340147
Epoch: 0297 cost = 0.532018880
Validation Loss: 0.531975
Epoch: 0298 cost = 0.529762844
Validation Loss: 0.53266215
Epoch: 0299 cost = 0.527516544
Validation Loss: 0.5317322
Epoch: 0300 cost = 0.525279740
Validation Loss: 0.52855545
Epoch: 0301 cost = 0.523052335
Validation Loss: 0.51883143
Epoch: 0302 cost = 0.520834267
Validation Loss: 0.51533854
Epoch: 0303 cost = 0.518625786
Validation Loss: 0.5165418
Epoch: 0304 cost = 0.516426732
Validation Loss: 0.51852
Epoch: 0305

Epoch: 0428 cost = 0.304509223
Validation Loss: 0.30065903
Epoch: 0429 cost = 0.303205152
Validation Loss: 0.29707265
Epoch: 0430 cost = 0.301905970
Validation Loss: 0.29627997
Epoch: 0431 cost = 0.300612301
Validation Loss: 0.29632822
Epoch: 0432 cost = 0.299323877
Validation Loss: 0.29618505
Epoch: 0433 cost = 0.298040529
Validation Loss: 0.29534012
Epoch: 0434 cost = 0.296762387
Validation Loss: 0.29414973
Epoch: 0435 cost = 0.295489401
Validation Loss: 0.29395545
Epoch: 0436 cost = 0.294221347
Validation Loss: 0.29074138
Epoch: 0437 cost = 0.292958657
Validation Loss: 0.28913534
Epoch: 0438 cost = 0.291701088
Validation Loss: 0.28920886
Epoch: 0439 cost = 0.290448387
Validation Loss: 0.2874667
Epoch: 0440 cost = 0.289200957
Validation Loss: 0.28575304
Epoch: 0441 cost = 0.287958364
Validation Loss: 0.28551695
Epoch: 0442 cost = 0.286720728
Validation Loss: 0.28496557
Epoch: 0443 cost = 0.285488149
Validation Loss: 0.28460285
Epoch: 0444 cost = 0.284260407
Validation Loss: 0.2824303

Epoch: 0568 cost = 0.163842435
Validation Loss: 0.16251755
Epoch: 0569 cost = 0.163083161
Validation Loss: 0.1609858
Epoch: 0570 cost = 0.162326823
Validation Loss: 0.15966812
Epoch: 0571 cost = 0.161573236
Validation Loss: 0.15935043
Epoch: 0572 cost = 0.160822161
Validation Loss: 0.15840028
Epoch: 0573 cost = 0.160074030
Validation Loss: 0.15889741
Epoch: 0574 cost = 0.159328560
Validation Loss: 0.15959546
Epoch: 0575 cost = 0.158585876
Validation Loss: 0.15847605
Epoch: 0576 cost = 0.157845880
Validation Loss: 0.15717302
Epoch: 0577 cost = 0.157108602
Validation Loss: 0.15690807
Epoch: 0578 cost = 0.156374007
Validation Loss: 0.1565341
Epoch: 0579 cost = 0.155642159
Validation Loss: 0.15628847
Epoch: 0580 cost = 0.154912782
Validation Loss: 0.15485051
Epoch: 0581 cost = 0.154186212
Validation Loss: 0.15322421
Epoch: 0582 cost = 0.153462196
Validation Loss: 0.15262899
Epoch: 0583 cost = 0.152740901
Validation Loss: 0.15254928
Epoch: 0584 cost = 0.152022146
Validation Loss: 0.1526243


...

Epoch: 4284 cost = 0.000267362
Validation Loss: 0.011669952
Epoch: 4285 cost = 0.000267352
Validation Loss: 0.008004264
Epoch: 4286 cost = 0.000267155
Validation Loss: 0.007733226
Epoch: 4287 cost = 0.000267217
Validation Loss: 0.010042429
Epoch: 4288 cost = 0.000266992
Validation Loss: 0.010367336
Epoch: 4289 cost = 0.000267103
Validation Loss: 0.011392198
Epoch: 4290 cost = 0.000266844
Validation Loss: 0.009931179
Epoch: 4291 cost = 0.000266987
Validation Loss: 0.008985404
Epoch: 4292 cost = 0.000266731
Validation Loss: 0.011328475
Epoch: 4293 cost = 0.000266938
Validation Loss: 0.01304291
Epoch: 4294 cost = 0.000266691
Validation Loss: 0.011410732
Epoch: 4295 cost = 0.000266878
Validation Loss: 0.0094220275
Epoch: 4296 cost = 0.000266673
Validation Loss: 0.009470189
Epoch: 4297 cost = 0.000266874
Validation Loss: 0.009926079
Epoch: 4298 cost = 0.000266618
Validation Loss: 0.009320471
Epoch: 4299 cost = 0.000266852
Validation Loss: 0.007027593
Epoch: 4300 cost = 0.000266632
Validatio

Epoch: 4438 cost = 0.000266323
Validation Loss: 0.005857825
Epoch: 4439 cost = 0.000266542
Validation Loss: 0.008452526
Epoch: 4440 cost = 0.000266352
Validation Loss: 0.011123304
Epoch: 4441 cost = 0.000266566
Validation Loss: 0.010546782
Epoch: 4442 cost = 0.000266363
Validation Loss: 0.007431184
Epoch: 4443 cost = 0.000266544
Validation Loss: 0.0067569437
Epoch: 4444 cost = 0.000266385
Validation Loss: 0.0043439325
Epoch: 4445 cost = 0.000266515
Validation Loss: 0.005411375
Epoch: 4446 cost = 0.000266414
Validation Loss: 0.006887388
Epoch: 4447 cost = 0.000266573
Validation Loss: 0.0068726507
Epoch: 4448 cost = 0.000266449
Validation Loss: 0.0045535387
Epoch: 4449 cost = 0.000266548
Validation Loss: 0.0029568372
Epoch: 4450 cost = 0.000266483
Validation Loss: 0.0022501478
Epoch: 4451 cost = 0.000266564
Validation Loss: 0.005883036
Epoch: 4452 cost = 0.000266541
Validation Loss: 0.007816086
Epoch: 4453 cost = 0.000266555
Validation Loss: 0.0059728483
Epoch: 4454 cost = 0.000266584
Va

Epoch: 4596 cost = 0.000266379
Validation Loss: 0.0055802977
Epoch: 4597 cost = 0.000266160
Validation Loss: 0.005963878
Epoch: 4598 cost = 0.000266392
Validation Loss: 0.0058740517
Epoch: 4599 cost = 0.000266199
Validation Loss: 0.008198404
Epoch: 4600 cost = 0.000266443
Validation Loss: 0.0102565745
Epoch: 4601 cost = 0.000266277
Validation Loss: 0.0082015935
Epoch: 4602 cost = 0.000266528
Validation Loss: 0.0047560926
Epoch: 4603 cost = 0.000266300
Validation Loss: 0.0031242294
Epoch: 4604 cost = 0.000266556
Validation Loss: 0.004565977
Epoch: 4605 cost = 0.000266325
Validation Loss: 0.005777638
Epoch: 4606 cost = 0.000266653
Validation Loss: 0.007900129
Epoch: 4607 cost = 0.000266358
Validation Loss: 0.009229466
Epoch: 4608 cost = 0.000266634
Validation Loss: 0.0054238085
Epoch: 4609 cost = 0.000266319
Validation Loss: 0.0034971912
Epoch: 4610 cost = 0.000266555
Validation Loss: 0.0069447854
Epoch: 4611 cost = 0.000266190
Validation Loss: 0.0058085597
Epoch: 4612 cost = 0.000266402

Epoch: 4755 cost = 0.000266511
Validation Loss: 0.0057891035
Epoch: 4756 cost = 0.000266087
Validation Loss: 0.006633339
Epoch: 4757 cost = 0.000266418
Validation Loss: 0.008078171
Epoch: 4758 cost = 0.000266043
Validation Loss: 0.0056081894
Epoch: 4759 cost = 0.000266285
Validation Loss: 0.005366901
Epoch: 4760 cost = 0.000265947
Validation Loss: 0.006760886
Epoch: 4761 cost = 0.000266200
Validation Loss: 0.005267259
Epoch: 4762 cost = 0.000265787
Validation Loss: 0.0069658896
Epoch: 4763 cost = 0.000266159
Validation Loss: 0.0055151535
Epoch: 4764 cost = 0.000265884
Validation Loss: 0.005568429
Epoch: 4765 cost = 0.000266139
Validation Loss: 0.009074834
Epoch: 4766 cost = 0.000265861
Validation Loss: 0.009762316
Epoch: 4767 cost = 0.000266134
Validation Loss: 0.008936349
Epoch: 4768 cost = 0.000265913
Validation Loss: 0.008186077
Epoch: 4769 cost = 0.000266121
Validation Loss: 0.004909757
Epoch: 4770 cost = 0.000265890
Validation Loss: 0.0039302493
Epoch: 4771 cost = 0.000266134
Vali

Validation Loss: 0.00615037
Epoch: 4912 cost = 0.000265776
Validation Loss: 0.007880478
Epoch: 4913 cost = 0.000265680
Validation Loss: 0.0064177834
Epoch: 4914 cost = 0.000265730
Validation Loss: 0.006184541
Epoch: 4915 cost = 0.000265670
Validation Loss: 0.0045551686
Epoch: 4916 cost = 0.000265668
Validation Loss: 0.0034443575
Epoch: 4917 cost = 0.000265594
Validation Loss: 0.0034373254
Epoch: 4918 cost = 0.000265642
Validation Loss: 0.0032882644
Epoch: 4919 cost = 0.000265572
Validation Loss: 0.00788174
Epoch: 4920 cost = 0.000265633
Validation Loss: 0.011755322
Epoch: 4921 cost = 0.000265568
Validation Loss: 0.010082429
Epoch: 4922 cost = 0.000265665
Validation Loss: 0.006369396
Epoch: 4923 cost = 0.000265514
Validation Loss: 0.004292784
Epoch: 4924 cost = 0.000265607
Validation Loss: 0.00659034
Epoch: 4925 cost = 0.000265527
Validation Loss: 0.008250462
Epoch: 4926 cost = 0.000265599
Validation Loss: 0.007409451
Epoch: 4927 cost = 0.000265478
Validation Loss: 0.0060676434
Epoch: 4

In [56]:
import csv
with open(log_files_path + '1layer13-9.csv', 'w',newline='') as myfile:
     wr = csv.writer(myfile)
     for i in range(len(togive2)):
        wr.writerow(list(togive2[i][0]))